# Performance benchmarks
This notebook takes the test dataset for the relevance and kpi models and runs the evaluation. The output of this notebook are the f1 scores for relevance and kpi models. 

In [1]:
import config
import os
import pathlib
import pandas as pd
from dotenv import load_dotenv
import json
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from src.data.s3_communication import S3Communication, S3FileType
from farm.infer import Inferencer
import zipfile
from farm.infer import QAInferencer
from farm.data_handler.utils import write_squad_predictions
from farm.evaluation import squad_evaluation

10/07/2022 19:33:17 - INFO - matplotlib.font_manager -   generated new fontManager
10/07/2022 19:33:18 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
# Load credentials
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [3]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url=os.getenv("S3_ENDPOINT"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    s3_bucket=os.getenv("S3_BUCKET"),
)

## Retrieve the test dataset and the trained models

In [4]:
s3c.download_files_in_prefix_to_dir(
    config.BASE_TRAIN_TEST_DATASET_S3_PREFIX,
    config.BASE_PROCESSED_DATA)

In [5]:
model_root = pathlib.Path(str(config.DATA_FOLDER)+"/models")
if not os.path.exists(model_root):
    model_root.mkdir(parents=True, exist_ok=True)
model_rel_zip = pathlib.Path(model_root, 'RELEVANCE.zip')
s3c.download_file_from_s3(model_rel_zip, config.CHECKPOINT_S3_PREFIX, "RELEVANCE.zip")

with zipfile.ZipFile(pathlib.Path(model_root, 'RELEVANCE.zip'), 'r') as z:
    z.extractall(model_root)

In [6]:
model_root = pathlib.Path(str(config.DATA_FOLDER)+"/models")
if not os.path.exists(model_root):
    model_root.mkdir(parents=True, exist_ok=True)
model_rel_zip = pathlib.Path(model_root, 'KPI_EXTRACTION.zip')
s3c.download_file_from_s3(model_rel_zip, config.CHECKPOINT_S3_PREFIX, 'KPI_EXTRACTION.zip')

with zipfile.ZipFile(pathlib.Path(model_root, 'KPI_EXTRACTION.zip'), 'r') as z:
    z.extractall(model_root)

# Relevance performance scores

In [7]:
test_data_path = str(config.BASE_PROCESSED_DATA)+'/rel_test_split.csv'
test_data = pd.read_csv(test_data_path, index_col=0)
test_data.head()

,label,text,text_b
1531,1,What is the total amount of direct greenhouse ...,"Without any regulatory mandates, our system's ..."
110,1,In which year was the annual report or the sus...,BP Sustainability Report 2018
2083,0,What is the total volume of natural gas liquid...,It is true that coal produces a greater volume...
648,0,What is the climate commitment scenario consid...,"ability. Our new Sustainability Report, prepar..."
744,0,What is the company name?,Each of our regulatory agreements can include ...


In [8]:
# get predictions from current model
model = Inferencer.load(str(model_root)+'/RELEVANCE')

result = model.inference_from_file(test_data_path)
results = [d for r in result for d in r["predictions"]]
preds = [int(r["label"]) for r in results]

test_data["pred"] = preds

10/07/2022 19:33:35 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
10/07/2022 19:33:38 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
10/07/2022 19:33:38 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
10/07/2022 19:33:38 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
10/07/2022 19:33:38 - INFO - farm.modeling.prediction_head -   Loading prediction head from /opt/app-root/data/models/RELEVANCE/prediction_head_0.bin
10/07/2022 19:33:39 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'RobertaTokenizer'
10/07/2022 19:33:39 - INFO - farm.data_handler.processor -   Initialized processor without tasks. Supply `metric` and `label_list` to the constructor for using the default task or add a custom task later via processor.add_task()
10/07/2022 19:33:39 - INFO - farm.utils -   devi

In [9]:
#evalute performance
groups = test_data.groupby("text")
scores = {}
for group, data in groups:
    pred = data.pred
    true = data.label
    scores[group] = {}
    scores[group]["accuracy"] = accuracy_score(true, pred)
    scores[group]["f1_score"] = f1_score(true, pred)
    scores[group]["recall_score"] = recall_score(true, pred)
    scores[group]["precision_score"] = precision_score(true, pred)
    scores[group]["support"] = len(pred)

/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# kpi wise performance metrics
scores_df = pd.DataFrame(scores)
scores_df.head()

,In which year was the annual report or the sustainability report published?,What is the annual total production from coal?,What is the base year for carbon reduction commitment?,What is the climate commitment scenario considered?,What is the company name?,What is the target carbon reduction in percentage?,What is the target year for climate commitment?,What is the total amount of direct greenhouse gases emissions referred to as scope 1 emissions?,What is the total amount of energy indirect greenhouse gases emissions referred to as scope 2 emissions?,What is the total amount of scope 1 and 2 greenhouse gases emissions?,...,What is the total amount of upstream energy indirect greenhouse gases emissions referred to as scope 3 emissions?,What is the total installed capacity from coal?,What is the total installed capacity from lignite (brown coal)?,What is the total volume of crude oil liquid production?,What is the total volume of hydrocarbons production?,What is the total volume of natural gas liquid production?,What is the total volume of natural gas production?,What is the total volume of proven and probable hydrocarbons reserves?,What is the volume of estimated probable hydrocarbons reserves?,What is the volume of estimated proven hydrocarbons reserves?
accuracy,0.891304,1.0,0.960000,0.960000,0.924528,0.970588,0.978261,1.0,1.0,1.0,...,1.0,0.750000,1.0,1.0,0.957143,1.0,0.937500,0.969697,1.0,1.0
f1_score,0.871795,1.0,0.960000,0.952381,0.900000,0.960000,0.975610,1.0,1.0,1.0,...,1.0,0.800000,0.0,1.0,0.949153,1.0,0.888889,0.969697,1.0,1.0
recall_score,0.871795,1.0,1.000000,0.909091,0.900000,1.000000,1.000000,1.0,1.0,1.0,...,1.0,0.666667,0.0,1.0,1.000000,1.0,0.800000,0.941176,1.0,1.0
precision_score,0.871795,1.0,0.923077,1.000000,0.900000,0.923077,0.952381,1.0,1.0,1.0,...,1.0,1.000000,0.0,1.0,0.903226,1.0,1.000000,1.000000,1.0,1.0
support,92.000000,6.0,25.000000,25.000000,53.000000,34.000000,46.000000,18.0,13.0,2.0,...,15.0,4.000000,1.0,3.0,70.000000,4.0,16.000000,33.000000,1.0,47.0


In [11]:
scores_df.loc['f1_score'].mean()

0.9155963800586726

# Kpi extraction performance scores

In [12]:
test_data_path = str(config.BASE_PROCESSED_DATA)+'/kpi_test_split.json'
test_data = pd.read_json(test_data_path)
test_data.head()

,version,data
0,v2.0,{'title': 'VERBUND-Integrated-Annual-Report-20...
1,v2.0,"{'title': 'FUGRO_JV2019_Clickable.pdf', 'parag..."
2,v2.0,"{'title': 'Enel SpA Annual Report 2019.pdf', '..."
3,v2.0,"{'title': 'Sustainability Report 2014_EN.pdf',..."
4,v2.0,"{'title': 'RWE-annual-report-2019.pdf', 'parag..."


In [13]:
kpi_df = s3c.download_df_from_s3(
    "aicoe-osc-demo/kpi_mapping",
    "kpi_mapping.csv",
    filetype=S3FileType.CSV,
    header=0,
)
kpi_df.head()

,kpi_id,question,sectors,add_year,kpi_category,Unnamed: 5,Unnamed: 6
0,0.0,What is the company name?,"OG, CM, CU",False,TEXT,NaN,NaN
1,1.0,In which year was the annual report or the sus...,"OG, CM, CU",False,TEXT,NaN,NaN
2,2.0,What is the total volume of proven and probabl...,OG,True,"TEXT, TABLE",NaN,NaN
3,2.1,What is the volume of estimated proven hydroca...,OG,True,"TEXT, TABLE",NaN,NaN
4,2.2,What is the volume of estimated probable hydro...,OG,True,"TEXT, TABLE",NaN,NaN


In [14]:
model = QAInferencer.load(str(model_root)+'/KPI_EXTRACTION', batch_size=40, gpu=True)

10/07/2022 19:35:29 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
10/07/2022 19:35:37 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
10/07/2022 19:35:37 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
10/07/2022 19:35:37 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [1024, 2]
10/07/2022 19:35:37 - INFO - farm.modeling.prediction_head -   Loading prediction head from /opt/app-root/data/models/KPI_EXTRACTION/prediction_head_0.bin
10/07/2022 19:35:40 - INFO - farm.modeling.tokenization -   Loading to

In [15]:
# run inference on validation dataset
results = model.inference_from_file(
    file=test_data_path,
    return_json=False,
)
result_squad = [x.to_squad_eval() for x in results]

preds_data_path = str(config.BASE_PROCESSED_DATA)+'/preds.json'

write_squad_predictions(
    predictions=result_squad,
    predictions_filename=test_data_path,
    out_filename=preds_data_path,
)

Inferencing Samples: 100%|██████████| 15/15 [00:07<00:00,  1.93 Batches/s]
10/07/2022 19:36:17 - ERROR - farm.modeling.predictions -   Both start and end offsets should be 0: 
6, 6 with a no_answer. 
Inferencing Samples: 100%|██████████| 11/11 [00:05<00:00,  1.94 Batches/s]
10/07/2022 19:39:47 - INFO - farm.data_handler.utils -   Written Squad predictions to: /opt/app-root/data/processed/preds.json


In [16]:
with open(test_data_path) as f:
    dataset_json = json.load(f)
    dataset = dataset_json['data']

with open(preds_data_path) as f:
    preds = json.load(f)

# NOTE: in predictions, the keys are strings but need to be converted to ints
# if we want to use squad evaluation file provided by farm
preds = {int(k): v for k, v in preds.items()}

na_probs = {k: 0.0 for k in preds}

# maps qid to True/False
qid_to_has_ans = squad_evaluation.make_qid_to_has_ans(dataset)
has_ans_qids = [k for k, v in qid_to_has_ans.items() if v]
no_ans_qids = [k for k, v in qid_to_has_ans.items() if not v]

# get raw scores
exact_raw, f1_raw = squad_evaluation.get_raw_scores_extended(dataset, preds)

# apply thresholds
exact_thresh = squad_evaluation.apply_no_ans_threshold(
    exact_raw,
    na_probs,
    qid_to_has_ans,
    1,
)
f1_thresh = squad_evaluation.apply_no_ans_threshold(
    f1_raw,
    na_probs,
    qid_to_has_ans,
    1,
)

# create results dict
results_squad = squad_evaluation.make_eval_dict(exact_thresh, f1_thresh)
if has_ans_qids:
    has_ans_eval = squad_evaluation.make_eval_dict(exact_thresh, f1_thresh, qid_list=has_ans_qids)
    squad_evaluation.merge_eval(results_squad, has_ans_eval, 'HasAns')
if no_ans_qids:
    no_ans_eval = squad_evaluation.make_eval_dict(exact_thresh, f1_thresh, qid_list=no_ans_qids)
    squad_evaluation.merge_eval(results_squad, no_ans_eval, 'NoAns')

# covert to df
scores_df = pd.DataFrame(results_squad, index=[0])
scores_df

,exact,f1,total,HasAns_exact,HasAns_f1,HasAns_total,NoAns_exact,NoAns_f1,NoAns_total
0,99.088272,99.218777,16891,45.357143,53.229875,280,99.99398,99.99398,16611


# Conclusion
The f1 scores for the relevance and the kpi extraction models were printed respectively. Next, we are going to look at the time taken by these models and implement sparse models that can optimize performance and efficiency. 